In [13]:
from networks import *
from simulation import *
import numpy as np

In [14]:
g = gen_BA_graph(1000, 3)

In [24]:
print(run_SIS_simulation(g, 0.3, time_steps=500))
print(run_SIS_simulation(g, 0.3, time_steps=1000))
print(run_SIS_simulation(g, 0.3, time_steps=2000))

0.161
0.137
0.143


In [26]:
for lam in np.linspace(0, 1, 5):
    print(run_SIS_simulation(g, 0.3))

0.156
0.138
0.14
0.134
0.16
